## Lead / Lag Plots

In [1]:
import xarray as xr
import os
import numpy as np
import netCDF4 as nc
import pandas as pd
from scipy import stats 
from scipy.interpolate import griddata, RectBivariateSpline
import matplotlib.pyplot as plt 
import statistics
#import seaborn as sns
from tqdm.auto import tqdm 
import operator as op
import cartopy as cart
import matplotlib.ticker as mticker
#from mpl_toolkits.axes_grid1 import make_axes_locatable
import pyproj
#from mpl_toolkits.basemap import Basemap, interp
import matplotlib.colors as colors
from matplotlib.axes import Axes
from cartopy.mpl.geoaxes import GeoAxes
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from scipy.stats import kstest, mannwhitneyu, ks_2samp
GeoAxes._pcolormesh_patched = Axes.pcolormesh
import matplotlib.dates as mdates
import cartopy
import cartopy.feature as cfeature

In [ ]:
from datetime import timedelta
import datetime

In [ ]:
size = (15, 15)
longdis = 15
latdis = 15
resolution = '10m'

# Map generation Code
def generateMap(central_longitude=0):
    
    fig = plt.figure(figsize=size)
    ax = fig.add_subplot(111)
    ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=central_longitude))
    ax.coastlines()
    gl = ax.gridlines(ccrs.PlateCarree(), draw_labels=True, linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
    gl.xlabels_top = False
    gl.xlocator = mticker.FixedLocator((range(-180, 180, longdis)))
    gl.ylocator = mticker.FixedLocator((range(-90, 90, latdis)))
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    ax.add_feature(cart.feature.BORDERS)

# Loading in the Data
### Note: update according to your file path

In [ ]:
%cd ERA5/precip_dailytotal
#first re-grid on 360x180 grid with 1x1 gridcells
yearly_precip_regridded = xr.open_dataset('precip_daily_total_1979-2019_regrid.nc', engine='netcdf4')

In [ ]:
#convert ERA5 precipitation to mm
yearly_precip_regridded = yearly_precip_regridded.tp*1000

In [ ]:
%cd ..
%cd humid_heat\regridded_daymeans

In [ ]:
all_tw_regridded = xr.open_dataset('TW_daily_max_1979-2020_daymean_regridded.nc', engine='netcdf4')
#all_tw_regridded

In [ ]:
%cd ..
%cd ..
%cd ..
daily_precip_regrid = xr.open_dataset('NOAA_CPC/precip_daily_total_1979-2022_regrid.nc', engine='netcdf4')
#daily_precip_regrid

In [ ]:
#%cd RainHeatData
%cd ERA5/2mtemp_dailymean/processed
#2mtemp_dailymax from ERA5
temp_dailymean = xr.open_dataset('2mtemp_dailymean_1979-2019_regridded.nc', engine='netcdf4')
#temp_dailymean

In [ ]:
%cd ..
%cd ..
%cd specific_humidity_dailymean

In [ ]:
spechum_dailymean = xr.open_dataset('spechum_dailymean_1979-2019_regrid.nc', engine='netcdf4')
#spechum_dailymean

In [ ]:
#vertical velocity
%cd ..
vv = xr.open_dataset('vertical_velocity_500hPa_1979-2023_regridded.nc',engine='netcdf4')

In [ ]:
#soil moisture
soil = xr.open_dataset('swv_dailymean_1979-2021_regridded.nc',engine='netcdf4')

In [ ]:
#ERA5 Northern Hemisphere
lats = np.argwhere((yearly_precip_regridded.lat.values >= 0) & (yearly_precip_regridded.lat.values <= 90))
lons = np.argwhere((yearly_precip_regridded.lon.values >= 0) & (yearly_precip_regridded.lon.values <= 360))
precip_data_N = yearly_precip_regridded.sel(lon = slice(0,360), lat=slice(0,90)) #yearly_precip_regridded
TW_data_N = all_tw_regridded.sel(lon = slice(0,360), lat=slice(0,90))
soil_N = soil.sel(lon = slice(0,360), lat=slice(0,90))
#temp_data_N = temp_dailymean.sel(lon = slice(0,360), lat=slice(0,90))
#spechum_data_N = spechum_dailymean.sel(lon = slice(0,360), lat=slice(0,90))
#vv_data_N = vv.sel(lon = slice(0,360), lat=slice(0,90))

#Southern Hemisphere
#lats = np.argwhere((yearly_precip_regridded.lat.values >= -90) & (yearly_precip_regridded.lat.values <= 0))
#lons = np.argwhere((yearly_precip_regridded.lon.values >= 0) & (yearly_precip_regridded.lon.values <= 360))
#precip_data_S = yearly_precip_regridded.sel(lon = slice(0,360), lat=slice(-90,0)) #yearly_precip_regridded
#TW_data_S = all_tw_regridded.sel(lon = slice(0,360), lat=slice(-90,0))
#temp_data = temp_dailymean.sel(lon = slice(0,360), lat=slice(-90,0))

In [ ]:
june_months = np.argwhere((yearly_precip_regridded.time.values.astype('datetime64[M]').astype(int) % 12 + 1) == 6)
july_months = np.argwhere((yearly_precip_regridded.time.values.astype('datetime64[M]').astype(int) % 12 + 1) == 7)
august_months = np.argwhere((yearly_precip_regridded.time.values.astype('datetime64[M]').astype(int) % 12 + 1) == 8)
JJA_idx = np.concatenate([june_months, july_months, august_months])
JJA_idx

december_months = np.argwhere((yearly_precip_regridded.time.values.astype('datetime64[M]').astype(int) % 12 + 1) == 12)
january_months = np.argwhere((yearly_precip_regridded.time.values.astype('datetime64[M]').astype(int) % 12 + 1) == 1)
february_months = np.argwhere((yearly_precip_regridded.time.values.astype('datetime64[M]').astype(int) % 12 + 1) == 2)
DJF_idx = np.concatenate([december_months, january_months, february_months])

In [ ]:
#select summer months
precip_data_JJA = precip_data_N.where(precip_data_N.time.dt.month.isin([6,7,8]),drop=True) #tp for ERA5, precip for NOAA
TW_data_JJA = TW_data_N.where(TW_data_N.time.dt.month.isin([6,7,8]),drop=True).TW
#temp_data_JJA = temp_data_N.where(temp_data_N.time.dt.month.isin([6,7,8]),drop=True).t2m
#spechum_data_JJA = spechum_data_N.where(spechum_data_N.time.dt.month.isin([6,7,8]),drop=True).q
#vv_data_JJ = vv.where(vv_data_N.time.dt.month.isin([6,7,8]),drop=True).w
soil_data_JJ = soil.where(soil_N.time.dt.month.isin([6,7,8]),drop=True).swvl1

In [ ]:
### select data for 1 x 1 grid points ###
## ONLY IF DOING ONE POINT SELECTION

#input location coordinates: Phoenix (33.5, 248.5)
input_lon=248.5
input_lat=33.5

precip_data = precip_data_JJA.sel(lon =slice(input_lon-.1,input_lon+.1), lat=slice(input_lat-.1,input_lat+.1))
precip_data_year = precip_data_N.sel(lon =slice(input_lon-.1,input_lon+.1), lat=slice(input_lat-.1,input_lat+.1))
#.mean(dim=['lat','lon'])
TW_data = TW_data_N.sel(lon =slice(input_lon-.1,input_lon+.1), lat=slice(input_lat-.1,input_lat+.1)).TW
#.mean(dim=['lat','lon'])
temp_data = temp_data_N.sel(lon =slice(input_lon-.1,input_lon+.1), lat=slice(input_lat-.1,input_lat+.1)).t2m
spechum_data = spechum_data_N.sel(lon =slice(input_lon-.1,input_lon+.1), lat=slice(input_lat-.1,input_lat+.1)).q
#precip_data
vv_data = vv_data_N.sel(lon =slice(input_lon-.1,input_lon+.1), lat=slice(input_lat-.1,input_lat+.1)).w
soil_data = soil_N.sel(lon =slice(input_lon-.1,input_lon+.1), lat=slice(input_lat-.1,input_lat+.1)).swvl1

## 3 regions of focus
* Region 1: Southern Arizona: 32-34N, 243-250E (try 32-38N) -- NEW COORD: 32-37N, 243-248E
* Region 2: East James Bay to US-Canada Border: 48-54N, 270-290E
* Region 2 updated: 49-55N, 282-289 N
* Region 3: Calcutta & Dhaka: 22-24N, 87-90E
* Region 4 (supplemental): Morocco:  20-30N, 0-10E

In [ ]:
### select area to average over ###
# REGIONALLY AVERAGED SELECTIONS

#lats = np.argwhere((yearly_precip_regridded.lat.values >= 32.5) & (yearly_precip_regridded.lat.values <= 34.5))
#lons = np.argwhere((yearly_precip_regridded.lon.values >= 289.5) & (yearly_precip_regridded.lon.values <= 293.5))

lon_1=87.5
lon_2=90.5
lat_1=22.5
lat_2=24.5

precip_data = precip_data_JJA.sel(lon =slice(lon_1-.1,lon_2+.1), lat=slice(lat_1-.1,lat_2+.1))
precip_data_year = precip_data_N.sel(lon =slice(lon_1-.1,lon_2+.1), lat=slice(lat_1-.1,lat_2+.1)).mean(dim=['lat','lon'])
TW_data = TW_data_N.sel(lon =slice(lon_1-.1,lon_2+.1), lat=slice(lat_1-.1,lat_2+.1)).TW.mean(dim=['lat','lon'])
#temp_data = temp_data_N.sel(lon =slice(lon_1-.1,lon_2+.1), lat=slice(lat_1-.1,lat_2+.1)).t2m.mean(dim=['lat','lon'])
#spechum_data = spechum_data_N.sel(lon =slice(lon_1-.1,lon_2+.1), lat=slice(lat_1-.1,lat_2+.1)).q.mean(dim=['lat','lon'])
vv_data = vv_data_N.sel(lon =slice(lon_1-.1,lon_2+.1), lat=slice(lat_1-.1,lat_2+.1)).w.mean(dim=['lat','lon'])
soil_data = soil_N.sel(lon =slice(lon_1-.1,lon_2+.1), lat=slice(lat_1-.1,lat_2+.1)).swvl1.mean(dim=['lat','lon'])

In [ ]:
### Daily Precip Anomalies ###
#daily average
precip_mean = precip_data.groupby('time.dayofyear').mean(dim='time')
#30 day rolling mean
precip_rolling = precip_data.rolling(time = 30, center = True).mean('time')
#anomaly
precip_data_anom = precip_data.groupby('time.dayofyear') - precip_rolling.groupby('time.dayofyear').mean(dim='time')

### Daily Max TW Anomalies ###
#daily average
TW_mean = TW_data.groupby('time.dayofyear').mean(dim='time')
#rolling mean
TW_rolling = TW_data.rolling(time = 30, center = True).mean('time')

#anomalies -- with 5 day smoothed climatology and without
TW_anom = TW_data.groupby('time.dayofyear') - TW_rolling.groupby('time.dayofyear').mean(dim='time')

In [ ]:
#only select unique TW data
_, index = np.unique(TW_anom['time'], return_index = True)
TW_unique = TW_anom.isel(time=index)

In [ ]:
### Daily 2m Temperature Anomalies ###
#daily average
temp_mean = temp_data.groupby('time.dayofyear').mean(dim='time')
#rolling mean
temp_rolling = temp_data.rolling(time = 30, center = True).mean('time')

#anomalies -- with 5 day smoothed climatology and without
temp_data = temp_data.groupby('time.dayofyear') - temp_rolling.groupby('time.dayofyear').mean(dim='time')

### Daily Mean Specific Humidity Anomalies ###
#daily average
spechum_mean = spechum_data.groupby('time.dayofyear').mean(dim='time')
#rolling mean
spechum_rolling = spechum_data.rolling(time = 30, center = True).mean('time')

#anomalies -- with 5 day smoothed climatology and without
spechum_data = spechum_data.groupby('time.dayofyear') - spechum_rolling.groupby('time.dayofyear').mean(dim='time')

In [ ]:
#get abs magnitudes
vv_mag = vv_data
soil_mag = soil_data

In [ ]:
#vv
vv_mean = vv_data.groupby('time.dayofyear').mean(dim='time')
vv_rolling = vv_data.rolling(time = 30, center = True).mean('time')
vv_data = vv_data.groupby('time.dayofyear') - vv_rolling.groupby('time.dayofyear').mean(dim='time')

In [ ]:
#soil m
soil_mean = soil_data.groupby('time.dayofyear').mean(dim='time')
soil_rolling = soil_data.rolling(time = 30, center = True).mean('time')
soil_data = soil_data.groupby('time.dayofyear') - soil_rolling.groupby('time.dayofyear').mean(dim='time')

In [ ]:
#change if you want magnitude of anomaly
_, index = np.unique(vv_data['time'], return_index = True)
vv_unique = vv_data.isel(time=index)

In [ ]:
_, index = np.unique(soil_data['time'], return_index = True)
soil_unique = soil_data.isel(time=index)

In [ ]:
#90th percentile cutoffs for each variable
# for anomalies use variable_data
# for abs magnitude use variable_mean
percentile_cutoff_JJA = np.nanpercentile(np.squeeze(precip_data), q=90, axis=0)
TW_percentile_cutoff_JJA = np.nanpercentile(np.squeeze(TW_unique), q=90, axis=0)
#temp_percentile_cutoff_JJA = np.nanpercentile(np.squeeze(temp_data), q=90, axis=0)
#spechum_percentile_cutoff_JJA = np.nanpercentile(np.squeeze(spechum_data), q=90, axis=0)
#TW_percentile_cutoff_JJA
#vv_percentile_cutoff_JJA = np.nanpercentile(np.squeeze(vv_unique), q=90, axis=0)
soil_percentile_cutoff_JJA = np.nanpercentile(np.squeeze(soil_unique), q=90, axis=0)
soil_percentile_cutoff_JJA

In [ ]:
### percentile of wet days >1 mm for area averaged ###

raw_percentile_cutoff_JJA = np.zeros((precip_data_JJA.shape[1], precip_data_JJA.shape[2]))
precip_days=[]
rain_threshold = 1
for i in range(precip_data.shape[1]):
    
    for j in range(precip_data.shape[2]):
        
        precip_data_for_coordinate = precip_data[:,i,j]
        idx_norain = np.squeeze(np.argwhere(np.array(precip_data_for_coordinate) <= rain_threshold))
        precip_data_JJA_pc = precip_data_for_coordinate[~idx_norain] #this is still an xarray, all days for given coord that have precip>1mm rain
        raw_percentile_cutoff_JJA[i][j] = np.nanpercentile(precip_data_JJA_pc, q=90)
        idx_90 = np.squeeze(np.argwhere(np.array(precip_data_for_coordinate) > raw_percentile_cutoff_JJA[i][j])) # put into index array
        precip_90_pc = precip_data_for_coordinate[idx_90] # select precip anomalies for this index
        precip_90_idx = precip_90_pc.time.to_dataframe() # put into dataframe, list of dates

In [ ]:
TW_data_JJA.shape
TW_data_coords = TW_data_JJA.sel(lon =slice(lon_1-.1,lon_2+.1), lat=slice(lat_1-.1,lat_2+.1))

In [ ]:
### percentile of TW days ###

TW_percentile_cutoff_JJA = np.zeros((TW_data_JJA.shape[2], TW_data_JJA.shape[1]))
TW_days=[]
threshold = 0.001
for i in range(TW_data_coords.shape[2]):
    
    for j in range(TW_data_coords.shape[1]):
        
        TW_data_for_coordinate = TW_data_coords[:,j,i]
        idx_no = np.squeeze(np.argwhere(np.array(TW_data_for_coordinate) <= threshold))
        TW_data_JJA_pc = TW_data_for_coordinate[~idx_no] 
        #TW_percentile_cutoff_JJA[i][j] = np.nanpercentile(TW_data_JJA_pc, q=90)
        TW_percentile_cutoff_JJA[i][j] = np.nanpercentile(TW_data_for_coordinate, q=90)
        idx_90_TW = np.squeeze(np.argwhere(np.array(TW_data_for_coordinate) > TW_percentile_cutoff_JJA[i][j])) # put into index array
        #idx_90_TW = (np.argwhere(np.array(TW_data_for_coordinate) > TW_percentile_cutoff_JJA[i][j])) # put into index array
        #idx_90_TW = np.squeeze(np.array(TW_data_for_coordinate))
        TW_90_pc = TW_data_for_coordinate[idx_90_TW] # select TW anomalies for this index
        TW_90_idx = TW_90_pc.time.to_dataframe() # put into dataframe, list of dates

In [ ]:
### PRECIP ANOMALY DAYS INDEX ###
#precip_90_idx2 = precip_90_idx.iloc[:2]
#max_percentile_idx = np.unravel_index(raw_percentile_cutoff_JJA.argmax(), raw_percentile_cutoff_JJA.shape)
        
for index, row in precip_90_idx.iterrows():
            
    # Buffer before
    buffer = 5
    date_datetime = index
    #date_datetime = datetime.strptime(date,'%Y-%m-%d')
            #date_datetime = datetime.strptime(date,'%Y-%m-%d') #turn into datetime 
    for addition in range(-1*buffer, 0): #5 days before, put actual day
        backward = date_datetime + timedelta(days = addition)
        precip_days.extend([backward])
                
                #First day aka day of
    precip_days.extend([date_datetime])

    for addition in range(1, buffer+1): # 5 days after event
        forward = date_datetime + timedelta(days = addition)
        precip_days.extend([forward])
        
precip_days_list = list(precip_days)

#for ind, ts in enumerate(precip_days_list):
 #   precip_days_list[ind] = precip_days_list[ind].to_datetime #convert to datetime64
    
#for ind, ts in enumerate(precip_days_list):
#    precip_days_list[ind] = precip_days_list[ind].from_date

In [ ]:
### TW ANOMALY DAYS INDEX ###
        
for index, row in TW_90_idx.iterrows():
            
    # Buffer before
    buffer = 5
    date_datetime = index
    #date_datetime = datetime.strptime(date,'%Y-%m-%d')
            #date_datetime = datetime.strptime(date,'%Y-%m-%d') #turn into datetime 
    for addition in range(-1*buffer, 0): #5 days before, put actual day
        backward = date_datetime + timedelta(days = addition)
        TW_days.extend([backward])
                
                #First day aka day of
    TW_days.extend([date_datetime])

    for addition in range(1, buffer+1): # 5 days after event
        forward = date_datetime + timedelta(days = addition)
        TW_days.extend([forward])
        
TW_days_list = list(TW_days)

In [ ]:
### Select +5/-5 days of Precip data ### 

#precip_days_tp = precip_data.where(precip_data.time.isin(precip_days_list), drop=True)
#precip_days_tp = precip_data.sel(time=precip_days_list)

#ABS mag
#precip_days_tp = precip_data_year.sel(time=precip_days_list)
precip_days_TW = precip_data_year.sel(time=TW_days_list)

#anomalies
#precip_days_tp = precip_data_anom.sel(time=precip_days_list)
#precip_days_TW = precip_data_anom.sel(time=TW_days_list)

In [ ]:
### Select +5/-5 days of TW data ###

#TW_days_tp = TW_unique.where(TW_unique.time.isin(precip_days_list), drop=True)
#TW_days_tp = TW_unique.sel(time=precip_days_list)
TW_days_TW = TW_unique.sel(time=TW_days_list)

#TW_days_tp

In [ ]:
### Select +5/-5 days of Specific Humidity ###

#Spechum_days_tp = spechum_data.where(spechum_data.time.isin(precip_days_list), drop=True)
Spechum_days_tp = spechum_data.sel(time=precip_days_list)
Spechum_days_TW = spechum_data.sel(time=TW_days_list)

#Select +5/-5 days of dry bulb temperature days
#Temp_days_tp = temp_data.where(temp_data.time.isin(precip_days_list), drop=True)
Temp_days_tp = temp_data.sel(time=precip_days_list)
Temp_days_TW = temp_data.sel(time=TW_days_list)

In [ ]:
###vv and soil m select ###
# anomalies = vv_data
# daily mean = vv_mean
#abs mag = vv_mag

#vv_days_tp = vv_unique.sel(time=precip_days_list)
#vv_days_TW = vv_unique.sel(time=TW_days_list)


#soil_days_tp = soil_unique.sel(time=precip_days_list)
soil_days_TW = soil_unique.sel(time=TW_days_list)

In [ ]:
### convert to dataframe ###

# RUN THIS IF CONDITIONING ON PRECIP

TW_df = TW_days_tp.to_dataframe()
precip_df = precip_days_tp.to_dataframe()
#spechum_df = Spechum_days_tp.to_dataframe()
#temp_df = Temp_days_tp.to_dataframe()
precip_df.tp
#vv_df = vv_days_tp.to_dataframe()
soil_df = soil_days_tp.to_dataframe()

In [ ]:
# RUN THIS IF CONDITIONING ON TW
TW_df = TW_days_TW.to_dataframe()
precip_df = precip_days_TW.to_dataframe()
#spechum_df = Spechum_days_TW.to_dataframe()
#temp_df = Temp_days_TW.to_dataframe()
precip_df.tp
#vv_df = vv_days_TW.to_dataframe()
soil_df = soil_days_TW.to_dataframe()

In [ ]:
# Add onset number and day number to dataframe
buffer_range = 2*buffer + 1 # (number of days spanning each plot -- 5 before, 1 day of, 5 after) -- should be 11
num_breaks = len(TW_df)/buffer_range #num_breaks should be a whole number --length(df)/11 = chunks of 11 day events
break_count = np.arange(1,num_breaks+1,1) #breaks = events -- 1 to # of events list

#days_before = np.arange(-1*buffer, 1,1) 
#days_after = np.arange(1, buffer + 1, 1) 
#days = np.hstack((days_before,days_after))
# I think you can also just skip the four lines above and do: days = np.arange(-1*buffer, buffer + 1, 1)
days = np.arange(-1*buffer, buffer + 1, 1) #x axis (-5 to +5)

# Stack repeating list of the day numbers in a column and add an event count
#day_array = np.tile(days,int(num_breaks)) #Construct an array by repeating 'days' the number of times given by int(num_breaks)
day_array = np.tile(days,int(num_breaks)) #stack everytime you have an event, grouped by event #
onset_array = np.repeat(break_count,buffer_range) #column of event numbers

TW_df['Event Number'] = onset_array
TW_df['Day Number'] = day_array
TW_df

In [ ]:
# Average across all events 
group_mean = TW_df.groupby('Day Number').mean()

# Find standard devations across all events
group_std = TW_df.groupby('Day Number').std()

# Add +/- 1 std and +/- 2 std to dataframe columns
group_mean['up_2std'] = group_mean['TW'] + 2*group_std['TW']
group_mean['down_2std'] = group_mean['TW'] - 2*group_std['TW']
group_mean['up_1std'] = group_mean['TW'] + group_std['TW']
group_mean['down_1std'] = group_mean['TW'] - group_std['TW']

In [ ]:
### 'Wet Bulb Temperature Anomalies surrounding Extreme Precipitation: Region X'

labels = [f'Break {i}' for i in range(1, int(num_breaks)+1)]
labels.append('Average')

xticks = [-5,-4,-3,-2,-1,0,1,2,3,4,5]
xlabels = [-5,-4,-3,-2,-1,0,1,2,3,4,5]

yticks = [-5,-4,-3,-2,-1,0,1,2,3,4,5,6,7]
ylabels = [-5,-4,-3,-2,-1,0,1,2,3,4,5,6,7]

fig, ax = plt.subplots(figsize=(6,6), facecolor='w', edgecolor='k')

for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(20)

# Plot black line for overall composite mean
mean_line = group_mean.reset_index().plot(ax = ax, x = 'Day Number', y = 'TW', color = 'black', label = labels[-1], linewidth = 5, kind = 'line')

# Shade between +/- 1 std
shading = ax.fill_between(group_mean.index,group_mean['up_1std'],group_mean['down_1std'], color = 'k', alpha = 0.4)

# Horizontal line at y = 0
ax.axhline(y=0, color='gray', linestyle='--', linewidth = 3)
ax.axvline(x=0, color='gray', linestyle='--', linewidth = 3)

# Shade around extreme precip day
#ax.axvspan(-1, 1, alpha=0.25, color='grey')    

plt.xlabel('Time Before/After Event', fontsize = 20)
plt.ylabel('Wet Bulb Temperature Anomaly (℃)', fontsize = 20)
#plt.title('Wet Bulb Temperature Anomalies surrounding Extreme Precipitation: Region 4')
ax.get_legend().remove()

ax.set_xticks(xticks)
ax.set_xticklabels(xlabels)
ax.set_yticks(yticks)
ax.set_yticklabels(ylabels)

plt.show()

## Soil Moisture

In [ ]:
#if conditioned on precip
#soil_df = soil_days_tp.to_dataframe()

#if conditioned on TW
soil_df = soil_days_TW.to_dataframe()

In [ ]:
# Add onset number and day number to dataframe
soil_df['Event Number'] = onset_array #[0:len(temp_df),]
soil_df['Day Number'] = day_array #[0:len(temp_df),]

# Average across all breaks
group_mean_soil = soil_df.groupby('Day Number').mean()

In [ ]:
# Take standard deviation across all breaks
group_std_soil = soil_df.groupby('Day Number').std()

# Add +/- 1 and 2 std to dataframe
group_mean_soil['up_2std'] = group_mean_soil['swvl1'] + 2*group_std_soil['swvl1']
group_mean_soil['down_2std'] = group_mean_soil['swvl1'] - 2*group_std_soil['swvl1']
group_mean_soil['up_1std'] = group_mean_soil['swvl1'] + group_std_soil['swvl1']
group_mean_soil['down_1std'] = group_mean_soil['swvl1'] - group_std_soil['swvl1']

In [ ]:
labels = [f'Break {i}' for i in range(1, int(num_breaks)+1)]
labels.append('Average')

#yticks = [-.1,0,.05,.1,.15,.2]
#ylabels = [-.1,0,.05,.1,.15,.2]

yticks = [-.1,-.05,0,.05,.1]
ylabels = [-.1,-.05,0,.05,.1]

fig, ax = plt.subplots(figsize=(6,6), facecolor='w', edgecolor='k')

for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(20)

# Plot black line as mean over all composite events
mean_line = group_mean_soil.reset_index().plot(ax = ax, x = 'Day Number', y = 'swvl1', color = 'firebrick', label = labels[-1], linewidth = 5, kind = 'line')

# Shade +/- 1 std
shading = ax.fill_between(group_mean_soil.index,group_mean_soil['up_1std'],group_mean_soil['down_1std'], color = 'firebrick', alpha = 0.4)

# Add horizontal line at y = 0
ax.axhline(y=0, color='gray', linestyle='--', linewidth = 3)
ax.axvline(x=0, color='gray', linestyle='--', linewidth = 3)
  
# Shade over duration of monsoon break
#ax.axvspan(-1, 1, alpha=0.25, color='grey')    
plt.xlabel('Time Before/After Event', fontsize = 20)
plt.ylabel('Soil Moisture Anomaly (m$^3$/m$^3$)', fontsize = 20)
#plt.title('Dry Bulb Temperature Anomalies surrounding Extreme Precipitation: Region 4')
ax.get_legend().remove()

ax.set_xticks(xticks)
ax.set_xticklabels(xlabels)
ax.set_yticks(yticks)
ax.set_yticklabels(ylabels)

plt.show()

## Vertical Velocity

In [ ]:
# Convert to pandas dataframe
#vv_df = vv_days_tp.to_dataframe()
vv_df = vv_days_TW.to_dataframe()

In [ ]:
# Add onset number and day number to dataframe
vv_df['Event Number'] = onset_array #[0:len(temp_df),]
vv_df['Day Number'] = day_array #[0:len(temp_df),]

# Average across all breaks
group_mean_vv = vv_df.groupby('Day Number').mean()

In [ ]:
# Take standard deviation across all breaks
group_std_vv = vv_df.groupby('Day Number').std()

# Add +/- 1 and 2 std to dataframe
group_mean_vv['up_2std'] = group_mean_vv['w'] + 2*group_std_vv['w']
group_mean_vv['down_2std'] = group_mean_vv['w'] - 2*group_std_vv['w']
group_mean_vv['up_1std'] = group_mean_vv['w'] + group_std_vv['w']
group_mean_vv['down_1std'] = group_mean_vv['w'] - group_std_vv['w']

In [ ]:
labels = [f'Break {i}' for i in range(1, int(num_breaks)+1)]
labels.append('Average')

yticks = [-.2,-.15,-.1,-.05,0,.05,.1,.15,.2]
ylabels = [-.2,-.15,-.1,-.05,0,.05,.1,.15,.2]

fig, ax = plt.subplots(figsize=(6,6), facecolor='w', edgecolor='k')

for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(20)

# Plot black line as mean over all composite events
mean_line = group_mean_vv.reset_index().plot(ax = ax, x = 'Day Number', y = 'w', color = 'steelblue', label = labels[-1], linewidth = 5, kind = 'line')

# Shade +/- 1 std
shading = ax.fill_between(group_mean_vv.index,group_mean_vv['up_1std'],group_mean_vv['down_1std'], color = 'steelblue', alpha = 0.4)

# Add horizontal line at y = 0
ax.axhline(y=0, color='gray', linestyle='--', linewidth = 3)
ax.axvline(x=0, color='gray', linestyle='--', linewidth = 3)
  
# Shade over duration of monsoon break
#ax.axvspan(-1, 1, alpha=0.25, color='grey')    
plt.xlabel('Time Before/After Event', fontsize = 20)
plt.ylabel('Vertical Velocity Anomaly (Pa/s)', fontsize = 20) #Vertical Velocity (Pa/s)
#plt.title('Dry Bulb Temperature Anomalies surrounding Extreme Precipitation: Region 4')
ax.get_legend().remove()

ax.set_xticks(xticks)
ax.set_xticklabels(xlabels)
ax.set_yticks(yticks)
ax.set_yticklabels(ylabels)

plt.show()

## Specific Humidity

In [ ]:
# Select daily mean specific humidity during extreme precip onset and surrounding
#Spechum_days_tp = spechum_data.where(spechum_data.time.isin(precip_days_list), drop=True)

# Convert to pandas dataframe
#SH_df = Spechum_days_tp.to_dataframe()
SH_df = Spechum_days_TW.to_dataframe()

In [ ]:
# Add onset number and day number to dataframe
SH_df['Event Number'] = onset_array[0:len(SH_df),]
SH_df['Day Number'] = day_array[0:len(SH_df),]

# Average across all breaks
group_mean_SH = SH_df.groupby('Day Number').mean()

SH_df.q = SH_df.q*1000
group_mean_SH.q = group_mean_SH.q*1000

# Find standard devations across all breaks
group_std_SH = SH_df.groupby('Day Number').std()

# Add +/- 1 std and +/- 2 std to dataframe columns
group_mean_SH['up_2std'] = group_mean_SH.q + 2*group_std_SH.q
group_mean_SH['down_2std'] = group_mean_SH.q - 2*group_std_SH.q
group_mean_SH['up_1std'] = group_mean_SH.q + group_std_SH.q
group_mean_SH['down_1std'] = group_mean_SH.q - group_std_SH.q

In [ ]:
labels = [f'Break {i}' for i in range(1, int(num_breaks)+1)]
labels.append('Average')

yticks = [-2,-1,0,1,2,3,4]
ylabels = [-2,-1,0,1,2,3,4]

fig, ax = plt.subplots(figsize=(6,6), facecolor='w', edgecolor='k')

for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(20)

# Plot black mean line
mean_line = group_mean_SH.reset_index().plot(ax = ax, x = 'Day Number', y = 'q', color = 'k', label = labels[-1], linewidth = 5, kind = 'line')

# Shade between +/- 1 std
shading = ax.fill_between(group_mean_SH.index,group_mean_SH['up_1std'],group_mean_SH['down_1std'], color = 'blue', alpha = 0.4)

# Add horizontal line for y = 0
ax.axhline(y=0, color='gray', linestyle='--', linewidth = 3)
ax.axvline(x=0, color='gray', linestyle='--', linewidth = 3)

# Shade break length
#ax.axvspan(-1, 1, alpha=0.25, color='grey')    
plt.xlabel('Time Before/After Event', fontsize = 20)
plt.ylabel('Specific Humidity Anomaly (g/kg)', fontsize = 20)
#plt.title('Specific Humidity Anomalies surrounding Extreme Precipitation: Region 4')
ax.get_legend().remove()

ax.set_xticks(xticks)
ax.set_xticklabels(xlabels)
ax.set_yticks(yticks)
ax.set_yticklabels(ylabels)

plt.show()

## Dry Bulb Temperature

In [ ]:
# Select daily max wet bulb temperatures during extreme precipitation onset and surrounding
#Temp_days_tp = temp_data.where(temp_data.time.isin(precip_days_list), drop=True)

# Convert to pandas dataframe
#temp_df = Temp_days_tp.to_dataframe()
temp_df = Temp_days_TW.to_dataframe()
#temp_df

In [ ]:
# Add onset number and day number to dataframe
temp_df['Event Number'] = onset_array #[0:len(temp_df),]
temp_df['Day Number'] = day_array #[0:len(temp_df),]

# Average across all breaks
group_mean_temp = temp_df.groupby('Day Number').mean()

In [ ]:
# Take standard deviation across all breaks
group_std_temp = temp_df.groupby('Day Number').std()

# Add +/- 1 and 2 std to dataframe
group_mean_temp['up_2std'] = group_mean_temp['t2m'] + 2*group_std_temp['t2m']
group_mean_temp['down_2std'] = group_mean_temp['t2m'] - 2*group_std_temp['t2m']
group_mean_temp['up_1std'] = group_mean_temp['t2m'] + group_std_temp['t2m']
group_mean_temp['down_1std'] = group_mean_temp['t2m'] - group_std_temp['t2m']

In [ ]:
labels = [f'Break {i}' for i in range(1, int(num_breaks)+1)]
labels.append('Average')

yticks = [-5,-4,-3,-2,-1,0,1,2,3,4,5]
ylabels = [-5,-4,-3,-2,-1,0,1,2,3,4,5]

fig, ax = plt.subplots(figsize=(6,6), facecolor='w', edgecolor='k')

for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(20)

# Plot black line as mean over all composite events
mean_line = group_mean_temp.reset_index().plot(ax = ax, x = 'Day Number', y = 't2m', color = 'k', label = labels[-1], linewidth = 5, kind = 'line')

# Shade +/- 1 std
shading = ax.fill_between(group_mean_temp.index,group_mean_temp['up_1std'],group_mean_temp['down_1std'], color = 'red', alpha = 0.4)

# Add horizontal line at y = 0
ax.axhline(y=0, color='gray', linestyle='--', linewidth = 3)
ax.axvline(x=0, color='gray', linestyle='--', linewidth = 3)
  
# Shade over duration of monsoon break
#ax.axvspan(-1, 1, alpha=0.25, color='grey')    
plt.xlabel('Time Before/After Event', fontsize = 20)
plt.ylabel('Temperature Anomaly (℃)', fontsize = 20)
#plt.title('Dry Bulb Temperature Anomalies surrounding Extreme Precipitation: Region 4')
ax.get_legend().remove()

ax.set_xticks(xticks)
ax.set_xticklabels(xlabels)
ax.set_yticks(yticks)
ax.set_yticklabels(ylabels)

plt.show()

## Precipitation

In [ ]:
# Select daily max wet bulb temperatures during extreme precipitation onset and surrounding
#precip_days_tp = precip_data.where(precip_data.time.isin(precip_days_list), drop=True)

# Convert to pandas dataframe
#precip_df = precip_days_tp.to_dataframe()
precip_df = precip_days_TW.to_dataframe()

In [ ]:
# Add onset number and day number to dataframe
precip_df['Event Number'] = onset_array[0:len(precip_df),]
precip_df['Day Number'] = day_array[0:len(precip_df),]

# Average across all breaks
group_mean_precip = precip_df.groupby('Day Number').mean()

In [ ]:
# Take standard deviation across all breaks
group_std_precip = precip_df.groupby('Day Number').std()

# Add +/- 1 and 2 std to dataframe
group_mean_precip['up_2std'] = group_mean_precip['tp'] + 2*group_std_precip['tp']
group_mean_precip['down_2std'] = group_mean_precip['tp'] - 2*group_std_precip['tp']
group_mean_precip['up_1std'] = group_mean_precip['tp'] + group_std_precip['tp']
group_mean_precip['down_1std'] = group_mean_precip['tp'] - group_std_precip['tp']

In [ ]:
labels = [f'Break {i}' for i in range(1, int(num_breaks)+1)]
labels.append('Average')

yticks = [0,5,10,15,20]
ylabels = [0,5,10,15,20]

fig, ax = plt.subplots(figsize=(6,6), facecolor='w', edgecolor='k')

for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(20)

# Plot black line as mean over all composite events
mean_line = group_mean_precip.reset_index().plot(ax = ax, x = 'Day Number', y = 'tp', color = 'k', label = labels[-1], linewidth = 5, kind = 'line')

# Shade +/- 1 std
shading = ax.fill_between(group_mean_precip.index,group_mean_precip['up_1std'],group_mean_precip['down_1std'], color = 'green', alpha = 0.4)

# Add horizontal line at y = 0
ax.axhline(y=0, color='gray', linestyle='--', linewidth = 3)
ax.axvline(x=0, color='gray', linestyle='--', linewidth = 3)
  
# Shade over duration of monsoon break
#ax.axvspan(-1, 1, alpha=0.25, color='grey')    
plt.xlabel('Time Before/After Event', fontsize = 20)
plt.ylabel('Precipitation Anomaly (mm)', fontsize = 20)
#plt.title('Extreme Precipitation: Region 4')
ax.get_legend().remove()

ax.set_xticks(xticks)
ax.set_xticklabels(xlabels)
ax.set_yticks(yticks)
ax.set_yticklabels(ylabels)

plt.show()

## Precip and WBT

In [ ]:
from matplotlib.legend_handler import HandlerLine2D

fig, ax1 = plt.subplots(figsize=(6,6), facecolor='w', edgecolor='k')
ax2 = ax1.twinx()

yticks = [-5,0,5,10,15,20,25,30,35]
ylabels = [-5,0,5,10,15,20,25,30,35]

y1ticks = [-1,0,1,2,3,4,5,6,7]
y1labels = [-1,0,1,2,3,4,5,6,7]

for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(20)

# Plot black line as mean over all composite events
mean_line = group_mean_precip.reset_index().plot(ax = ax1, x = 'Day Number', y = 'tp', color = 'g', label = 'Mean Precipitation', linewidth = 5, kind = 'line')
label = labels[-1]

#temp mean line
#mean_line = group_mean_temp.reset_index().plot(ax = ax2, x = 'Day Number', y = 't2m', color = 'r', label = labels[-1], linewidth = 5, kind = 'line')

#specific humidity mean line
#mean_line = group_mean_SH.reset_index().plot(ax = ax2, x = 'Day Number', y = 'q', color = 'blue', label = labels[-1], linewidth = 5, kind = 'line')

#wet bulb temp mean line
mean_line2 = group_mean.reset_index().plot(ax = ax2, x = 'Day Number', y = 'TW', color = 'black', label = 'Mean TW', linewidth = 5, kind = 'line')

# Shade +/- 1 std
shading = ax1.fill_between(group_mean_precip.index,group_mean_precip['up_1std'],group_mean_precip['down_1std'], color = 'green', alpha = 0.4)
shading = ax2.fill_between(group_mean.index,group_mean['up_1std'],group_mean['down_1std'], color = 'grey', alpha = 0.4)


# Add horizontal line at y = 0
#ax1.axhline(y=0, color='green', linestyle='--', linewidth = 3)
#ax2.axhline(y=0, color='k', linestyle='--', linewidth = 3)
  
labels = ["Mean tp","Average", "Mean TW"]
    
# Shade over duration of monsoon break
#ax1.axvspan(-1, 1, alpha=0.25, color='grey')   
ax1.axvline(x=0, color='gray', linestyle='--', linewidth = 3)
ax1.axhline(y=0, color='gray', linestyle='--', linewidth = 3)


ax1.set_xlabel('Time Before/After Event', fontsize = 16)
ax1.set_ylabel('Precipitation Anomaly (mm)', fontsize = 16)
ax1.legend("tp")
ax1.legend(['mean precip'], loc='upper left')
#ax1.set_yticks(yticks)
#ax1.set_yticklabels(ylabels)
ax1.get_legend().remove()
ax1.set_yticks(y1ticks)
ax1.set_yticklabels(y1labels,fontsize = 16)

ax2.set_ylabel('Wet Bulb Temperature Anomaly (℃)', fontsize = 16)
ax2.legend("TW")
ax2.legend(['mean TW'], loc='upper right')
ax2.get_legend().remove()
ax2.set_yticks(yticks)
ax2.set_yticklabels(ylabels,fontsize = 16)

ax1.set_xticks(xticks)
ax1.set_xticklabels(xlabels,fontsize = 16)

plt.show()

## Zoom in on Difference Map Region

In [ ]:
day_of_JJA = np.loadtxt('/RainHeatData/day_of_JJA.txt', delimiter = ',')
day_of_JJA_precip = np.loadtxt('/RainHeatData/day_of_JJA_precip.txt', delimiter = ',')

In [ ]:
lats = np.argwhere((precip_data_JJA.lat.values >= 0) & (precip_data_JJA.lat.values <= 90))
lons = np.argwhere((precip_data_JJA.lon.values >= 0) & (precip_data_JJA.lon.values <= 360))

fig= plt.figure(figsize=(14,6))
ax = plt.axes(projection=ccrs.PlateCarree())
extent = [240, 255, 30, 40] #15 X 10
cm = plt.cm.get_cmap('coolwarm')
res = '110m'

plt.rcParams.update({'font.size': 20})

ax.set_extent(extent)
#ax.gridlines()
ax.add_feature(cfeature.COASTLINE.with_scale(res))
ax.add_feature(cfeature.BORDERS.with_scale(res))
ax.add_feature(cfeature.LAND.with_scale(res))
ax.add_feature(cfeature.STATES.with_scale(res))

gl = ax.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False

#pcolormesh plot
sc = plt.pcolormesh(precip_data_JJA.lon.values[lons[:,0]], precip_data_JJA.lat.values[lats[:,0]], day_of_JJA,transform=ccrs.PlateCarree(), cmap='coolwarm') #coolwarm, BrBG
#norm=divnorm
#pcolormesh
#sm = plt.pcolormesh(precip_data_JJA.lon.values[lons[:,0]], precip_data_JJA.lat.values[lats[:,0]], np.ma.masked_less(day_of_DJF_sigmask, 0), facecolor = 'None', edgecolors = None)

# Add inset map
#from mpl_toolkits.axes_grid1.inset_locator import inset_axes
#axins = inset_axes(ax, width="60%", height="60%", loc='lower left',
                   #bbox_to_anchor=(-0.05, 0.2, 0.5, 0.5),
 #                  bbox_to_anchor=(.05, .2, .5, .5),
  #                 bbox_transform=ax.transAxes,
   #                axes_class=cartopy.mpl.geoaxes.GeoAxes,
    #               axes_kwargs=dict(map_projection=ccrs.PlateCarree()))

# Add land, state borders, coastline, and country borders to inset map
#axins.add_feature(cartopy.feature.LAND.with_scale(res), facecolor='k', alpha=0.8, zorder=0)
#axins.add_feature(cartopy.feature.COASTLINE.with_scale(res))
#axins.add_feature(cartopy.feature.BORDERS.with_scale(res))

# Set the lat/lon limits of the inset map [x0, x1, y0, y1]
inset_extent = [247.5, 249.5, 32.5, 34.5]
#ax.set_extent(inset_extent)

# Add box around location of inset map on the main map
x = [inset_extent[0], inset_extent[1], inset_extent[1], inset_extent[0], inset_extent[0]]
y = [inset_extent[2], inset_extent[2], inset_extent[3], inset_extent[3], inset_extent[2]]
ax.plot(x, y,color='blue', alpha=1.0, linewidth=6.0,transform=ccrs.PlateCarree())

# Draw lines between inset map and box on main map
#rect, connectors = ax.indicate_inset_zoom(axins, edgecolor="black", alpha=0.5, transform=ax.transAxes)
# By default only two of the connecting lines (connectors) are shown
# it is possible to choose which of the lines to show by setting the visibility
# connectors are counted clockwise from the lower-left corner
#connectors[0].set_visible(False)
#connectors[1].set_visible(True)
#connectors[3].set_visible(True)
#connectors[2].set_visible(True)

#ax.set_xticklabels(top_labels,fontsize=20)
#ax.set_yticklabels(right_labels,fontsize=20)

cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
#ax.set_title('Difference in Mean Extreme Wet Bulb Temperature Anomaly on Extreme Precipitation Anomaly Days (90%, JJA) vs All Days')
plt.colorbar(sc ,cax=cax) # Similar to fig.colorbar(im, cax = cax) sc, cax=cax
plt.clim(-2.5,2.5)
plt.rc('axes', labelsize=20)
#-2.5-2.5 for TW, -4-4 for Precip
#-5+5 for t2m, -8+8 for precip

#plt.savefig('/RainHeatData/FinalFigs/Region1_boxed_mean_TW_diff_dayofavg.png')
plt.show()